# Functional abstraction

_Note_: ensure that students copy, by hand and on paper, the various definitions written by the teacher on the whiteboard. It is strongly advised to ask students *not* to use a laptop, as it will prove distracting.

The topics discussed are:
- let us consider the following program
```
k := 0
x := 0
incr_x := <k => x := x + k>
incr_x(1)
incr_x(3)
...
```

- if we follow the semantics we have given in the previous lecture, then when we invoke `incr_x` for the first time then we redefine the value of `k`
    - this means that functions are abstracting code, but in a way that "leaks" into the context
    - leaks are very hard to manage, because they stop us from thinking about the function in isolation with respect to the rest of the program
    - moreover, we expect parameters such as `k` inside the function to have no relation with the parameters defined outside the function
- for this reason we introduce the notion of _scope_
    - variables defined _inside_ a function only exist while the function is being called, and must then be removed from the state when the function is done
    - if these variables have the same name as other variables in the program, the other variables are left untouched
    - such variables are called _local_ (to the function), because they only exist inside the function and do not leak out of it
    - other variables are called _global_
- scope is implemented, in the semantics, by adding yet another component to our state: the so-called _call stack_
    - the _call stack_ is a stack which will contain all the local variables of functions active at the current time
    - when a function is invoked, we add these values to the top of the stack
    - when the function is done, we remove the values from the top of the stack
- we define a new instruction: `call`, in order to denote and delimit the invocation of a function
    - the state also contains a new entry, `stack`, where the local variables are pushed upon invocation
        - the stack is either empty $\{ \}$, or it contains a head $h$, with the local variables, and a tail $t$ with the rest of the stack
            - for example: $\{ stack := \{ h := \{ x := 1 \}, t := \{ \} \}, x := 1 \}$ is a state where variable `x` is defined both at the top of the stack and in the regular state
- the new semantics of function invocation therefore becomes:
    
    $\text{eval}(<V(e_1, e_2, \dots)>, S) \rightarrow <call(a_1 := e_1; a_2 := e_2; \dots; L)>, S[\text{stack} := \{ h := \{ a_1 := \text{null}, a_2 := \text{null}, \dots \}, t := s[\text{stack}] \}]$ where $S[V] \rightarrow <(a_1, a_2, \dots) \Rightarrow L>$
    
- let us consider only instruction `incr_x := <k => x := x + k>; incr_x(1)` where the state is $\{ k := 0, x := 0 \}$
    - inside the body of the function, accessing `k` must now look in `S[stack][h][k]`, even though `S[k]` also exists
- this means that variable lookup is now divided into two rules, in order to first look at the head of the stack and then, if nothing is found, in the globals:
    - $\text{eval}(<V>, S) \rightarrow <S[\text{stack}][h][V]>, S$, where `V` is a variable name
    - $\text{eval}(<V>, S) \rightarrow <S[V]>, S$, where `V` is a variable name and $\not\exists S[\text{stack}][h][V]$
    - this implies that we have removed $eval-expr$, and are now using $eval$ for both statements and expressions
        - this has the important effect that evaluating expressions can now alter the state!
- when a function terminates, then its body becomes `done` and we reach the combination `call(done)`: then we must "pop the stack", that is remove the last level of local variables from the stack in order to cleanup the function
    - $eval(<call(done)>, S) \rightarrow <done>, S[stack := S[stack][t]]$
- let us see the original example, again, in depth, with the new lookup rules:

```
k := 0
x := 0
incr_x := <k => x := x + k>
incr_x(1)
incr_x(3)
...
```

```
title := "Mr."
name := "Strange"
add_title := <title => name := title + " " + name>
add_title("Dr.")
add_title(title)
```

[[[add more examples from the practicums with variable capture]]]

- with the stack, we can nest function calls arbitrarily: a function can call another function

```
k := 0
x := 0
double := <() => x := x + x>
quadruple := <() => double(); double()>
```

[[[add more examples from the practicums with functions calling other functions]]]

- even if some of these functions all use the same parameters, we still do not get clashes because each function stores its local functions in another location on the stack

```
k := 0
x := 0
incr_x := <k => x := x + k>
mult_x := <k => x := x * k>
mult_incr_x := <(k,c) => mult_x(k); incr_x(c)>
mult_incr_x(2,3)
...
```

[[[add more examples from the practicums with functions calling other functions and parameter shadowing]]]

- the block of code in which a variable is active is called its _scope_
- for example, in the following, there are four different scopes for `k`

```
k := 0
x := 0
incr_x := <k => x := x + k>
mult_x := <k => x := x * k>
mult_incr_x := <(k,c) => mult_x(k); incr_x(c)>
mult_incr_x(2,3)
...
```

[[[add some examples for drawing asterisks and lines, buildup to the hollow square]]]